In [1]:
import os, os.path
import pathlib
import random
import sys

import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib

In [2]:
os.getcwd()
INSPECTION_PATH = pathlib.Path(os.getcwd()).parent / "country-backend"
sys.path.append(str(INSPECTION_PATH.absolute()))

In [3]:
from country.model.predict import model
from country.model.predict import preprocess, prediction, predict_image

2022-02-24 08:56:07.261678: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
from country.explainer.new_lime_ import explain_image
from PIL import Image


In [ ]:
### adjustments for multiclass/binary model ###

# in predict.py change: 
# - ln.12 -> model 
# - choose (by comment out/in) desired decode_model_output method: multiclass or binary
# -> then load imports in jpyn again / restart jupyter kernel

In [ ]:
# To do notes #

# get more images for Images 3 - > Images 4 with 100 images for each city
# currently (Images3): only one false prediction on country basis 

In [ ]:
### Test 1: Explain images and save ###

In [9]:
#load images
images = []
names = []
labels = []

#images -> load according to which model is used
'''
paths = ["/Users/theresatroger/z_GTC_Results/Images2/Berlin", 	#for both multiclass and binary country model
			"/Users/theresatroger/z_GTC_Results/Images2/Hamburg", 
			"/Users/theresatroger/z_GTC_Results/Images2/Tel_Aviv", 
			"/Users/theresatroger/z_GTC_Results/Images2/Westjerusalem"]
'''
'''
paths = ["/Users/theresatroger/z_GTC_Results/Images2/Berlin", 	#for binary germany model
			"/Users/theresatroger/z_GTC_Results/Images2/Hamburg"]
'''
'''
paths = ["/Users/theresatroger/z_GTC_Results/Images2/Tel_Aviv", #for binary israel model
			"/Users/theresatroger/z_GTC_Results/Images2/Westjerusalem"]
'''
paths = ["/Users/theresatroger/z_GTC_Results/Umfrage_welches_Modell/Images/schwer/neu"]
pathresults = "/Users/theresatroger/z_GTC_Results/Umfrage_welches_Modell/explanations/20220224_explanations/" #results

for path in paths:
	for f in os.listdir(path):
		if f.endswith(".png"):
			images.append(cv2.imread(os.path.join(path,f)))
			names.append(f)
			labels.append(path.split("/")[-1])

#print(names)
#print (labels)	

for i, img in enumerate(images):
	#1 preprocess image and get prediction
	print (i)
	img = preprocess(img)
	prediction = predict_image(img)
	img = np.squeeze(img)
	print ("Prediction:", prediction)	
	print ("Label:", labels[i])	
    #2 create explanation
	image_explanation = explain_image(img = img, seg_method = "slic", seg_settings = {}, num_of_samples = 700, samples_p = 0.5,
					model_ = model, threshold = 0.5, volume = 20, colour =  "green", transparency = 0.7) 
	#3 save image
	#out_img = np.hstack((img,image_explanation))
	matplotlib.image.imsave(pathresults+str(i)+str(labels[i])+"_"+str(prediction)+".png", ((image_explanation+1)*127.5/255))
	

0
Prediction: Westjerusalem
Label: neu
1
Prediction: Hamburg
Label: neu
2
Prediction: Tel_Aviv
Label: neu
3
Prediction: Tel_Aviv
Label: neu


In [ ]:
### Test 2: Explaining images with certain prediction

In [ ]:
#load images
images = []
names = []
labels = []

#images -> load according to which model is used
'''
paths = ["/Users/theresatroger/z_GTC_Results/Images4/Berlin", 	#for both multiclass and binary country model
			"/Users/theresatroger/z_GTC_Results/Images4/Hamburg", 
			"/Users/theresatroger/z_GTC_Results/Images4/Tel_Aviv", 
			"/Users/theresatroger/z_GTC_Results/Images4/Westjerusalem"]
'''
paths = ["/Users/theresatroger/z_GTC_Results/Images4/Berlin", 	#for binary germany model
			"/Users/theresatroger/z_GTC_Results/Images4/Hamburg"]
'''
paths = ["/Users/theresatroger/z_GTC_Results/Images4/Tel_Aviv", #for binary israel model
			"/Users/theresatroger/z_GTC_Results/Images4/Westjerusalem"]
'''
pathresults = "/Users/theresatroger/z_GTC_Results/Test_Images4_falsepredictions/Results_germany/" #results

for path in paths:
	for f in os.listdir(path):
		if f.endswith(".png"):
			images.append(cv2.imread(os.path.join(path,f)))
			names.append(f)
			labels.append(path.split("/")[-1])


#print(names)
#print (labels)	
count_no_match = 0

for i, img in enumerate(images):
	#1 preprocess image
	img = preprocess(img)
	prediction = predict_image(img)
	img = np.squeeze(img)
	print (i)
	print ("Prediction:", prediction)	
	print ("Label:", labels[i])	

    #explain only images with certain prediction   -> adapt according to binary prediction output
	#if (prediction == "Israel" and labels[i] == ("Berlin" or "Hamburg")) or (prediction == "Germany" and labels[i] == ("Tel_Aviv" or "Westjerusalem")):
	if prediction != labels[i]: 
		print (i, "no match")
		count_no_match += 1
		#2 explain image
		image_explanation = explain_image(img = img, seg_method = "slic", seg_settings = {}, num_of_samples = 700, samples_p = 0.5,
					model_ = model, threshold = 0.5, volume = 20, colour =  "green", transparency = 0.7) 
		#3 save image
		out_img = np.hstack((img,image_explanation))
		matplotlib.image.imsave(pathresults+str(i)+str(labels[i])+"_"+str(prediction)+".png", ((out_img+1)*127.5/255)) #scale back from original scaling while preprocessing
	else: 
		print (i, "match")

print ("Total no matches:", count_no_match)

In [ ]:
### generate explanation for one image without saving ###

In [ ]:
img = cv2.imread("/Users/theresatroger/Desktop/Images/Westjerusalem1.png")
img = preprocess(img)
img = np.squeeze(img)


plt.imshow((img +1)*127.5/255)
plt.axis("off")
plt.show()

image_explanation = explain_image(img = img, seg_method = "slic", seg_settings = {}, num_of_samples = 700, samples_p = 0.5,
                  model_ = model, threshold = 0.5, volume = 20, colour =  "green", transparency = 0.7)

plt.imshow((image_explanation +1)*127.5/255)
plt.axis("off")
plt.show()



In [ ]:
### test binary prediciton ### 

In [ ]:
berlin1 = preprocess(cv2.imread("/Users/theresatroger/z_GTC_Results/Images1/Berlin/Berlin1.png"))
berlin2 = preprocess(cv2.imread("/Users/theresatroger/z_GTC_Results/Images1/Berlin/Berlin2.png"))
telaviv1 = preprocess(cv2.imread("/Users/theresatroger/z_GTC_Results/Images1/Tel_Aviv/TelAviv1.png"))
telaviv2 = preprocess(cv2.imread("/Users/theresatroger/z_GTC_Results/Images1/Tel_Aviv/TelAviv2.png"))
hamburg1 = preprocess(cv2.imread("/Users/theresatroger/z_GTC_Results/Images1/Hamburg/Hamburg1.png"))
hamburg2 = preprocess(cv2.imread("/Users/theresatroger/z_GTC_Results/Images1/Hamburg/Hamburg2.png"))
westjerusalem1 = preprocess(cv2.imread("/Users/theresatroger/z_GTC_Results/Images1/Westjerusalem/Westjerusalem1.png"))
westjerusalem2 = preprocess(cv2.imread("/Users/theresatroger/z_GTC_Results/Images1/Westjerusalem/Westjerusalem2.png"))

#pb1 = predict_image(berlin1)
#pb2 = predict_image(berlin2)
pt1 = predict_image(telaviv1)
pt2 = predict_image(telaviv2)
#ph1 = predict_image(hamburg1)
#ph2 = predict_image(hamburg2)
pw1 = predict_image(westjerusalem1)
pw2 = predict_image(westjerusalem2)
#print ("Berlin", pb1, pb2)
print ("Tel Aviv", pt1, pt2)
#print ("Hamburg", ph1, ph2)
print ("Westjerusalem", pw1, pw2)